In [1]:
import math
import numpy as np
import statistics
import matplotlib.pyplot as plt

In [6]:
# this function generates given no of random numbers from [0,1]
# Xo is the seed
# full period if m is a power of 2, c is odd and a is 4*n + 1
(a, c, m, x) = (2053, 1345, pow(2, 40), 3245)
(k, λ, σ) = (0.8, 2.9, 3)

def Uniform_Distribution():
    global x
    x = (a * x + c) % m 
    return x/m

def Weibull_Distribution():
    U = Uniform_Distribution()
    return np.power(-np.log(U),1/k)*σ

In [7]:
def f(S):
    Y=0
    for j in range(S): 
        Y += Weibull_Distribution() 
    if Y < 5:
        return 1
    else:
        return 0

def strata_stat(S, nj):
    Yij_store = []
    for i in range(nj):
        Yij = f(S)
        Yij_store.append(Yij)
    μj_hat = np.mean(Yij_store)
    sj = np.sqrt(statistics.variance(Yij_store))
    return μj_hat, sj


def poisson_PDF(S):
    return (math.exp(-λ)*(λ**(S)))/math.factorial(S) 

def strata_calc(S, nj):
    if S < 6:
        μj_hat, sj = strata_stat(S, nj)
    else:
        num = 1
        for i in range(5):
            num = num - poisson_PDF(i)
        mean_no_of_storms = math.floor(num/λ)

        μj_hat, sj = strata_stat(mean_no_of_storms, nj)
    return μj_hat, sj

def strata_prob_calc(S):
    if S <= 5:
        return poisson_PDF(S)
    else:
        prob = 1
        for i in range(6):
            prob = prob - poisson_PDF(i)
        return prob

In [8]:

for n in [100, 10000]:
    Var = 0
    μ_strat = 0
    others = 0
    for S in range(7):
        wj = strata_prob_calc(S)
        nj = int(n*wj)
        μj_hat, sj = strata_calc(S, nj)
        μ_strat += (wj)*μj_hat
        Var+=(np.power(wj, 2)*sj*sj)/nj
    print(f"99% confidence interval for N={n} is :({μ_strat-2.58*np.sqrt(Var)}, {μ_strat+2.58*np.sqrt(Var)})")

99% confidence interval for N=100 is :(0.349941893450217, 0.5671474456051656)
99% confidence interval for N=10000 is :(0.44043247167212546, 0.460387596994791)
